# SIATEC Patterns and Tune Families in The Session

### Abstract

In this notebook we will run one of the best-known musical pattern extraction algorithms on tunes from *The Session*, and implement a simple method for the task of tune family detection (similar to the task of cover-song detection).


### Introduction

As already described (Polifonia deliverables D3.1-D3.6, 2021-2024; Danny Diamond MSc thesis, 2024), the concept of tune families is an important one in the musicology of traditional music, and we have developed methods of automatically detecting tune families based on several types of $n$-gram melodic patterns. The goal of this notebook is to implement another method, based on a totally different definition of melodic patterns. As we will see, this method does not work well -- not because of a weakness in the algorithms or the patterns, but because this new definition of pattern has a different goal -- *compression* rather than *linking*.


### SIATEC patterns

SIATEC, COSIATEC, and SIATECCompress are members of a family of pattern-extraction algorithms developed by David Meredith in multiple papers over several years. A representative paper is: [*COSIATEC and SIATECCompress:
Pattern discovery by geometric compression*](https://vbn.aau.dk/en/publications/094ac3cf-78ca-4f87-8c6a-77303e3b1d36), David Meredith, Music Information Retrieval Evaluation eXchange (MIREX 2013).

These algorithms work on *point-set* data, ie music where each note is represented only by an onset time (in beats, or MIDI ticks, or another unit) and a pitch (chromatic or diatonic). The algorithms work by detecting subsets of the data which occur more than once. They aim to extract the best subsets, which occur most often, using heuristics such as "compactness" of the pattern, to help. When detected, the pattern is removed from the dataset, and each repetition is replaced by a reference. Each repetition that is detected and encoded in such an extracted pattern thus allows the original dataset to be compressed.


We focus in this notebook on the SIATECCompress algorithm, which allows patterns to overlap with each other. This choice is primarily because it is faster (Meredith, 2013) and our corpus is large.

The implemention we use is `OMNISIA.jar`, from: http://www.titanmusic.com/software.php. We are grateful to David Meredith for providing a high-quality implementation.

In [1]:
import os, subprocess
from pathlib import Path
from collections import defaultdict
import pandas as pd
import time
import pickle

### Pre-processing

We begin with the file [`thesession.abc`, available in GitHub](https://github.com/adactio/TheSession-data). For certainty and comparability, we host a specific dump under the Polifonia project:

https://github.com/polifonia-project/folk_ngram_analysis/blob/master/FoNN/thesession_corpus/abc/thesession.abc

We process this to MIDI files using `abc2midi.exe`, available from: https://ifdo.ca/~seymour/runabc/top.html 

In particular, https://ifdo.ca/~seymour/runabc/abcMIDI-2024.03.21.zip has `abc2midi.exe`.

The command line we use is:

`.\abc2midi.exe thesession.abc -silent -NGRA -NCOM`

This gives a `.mid` file for each tune in the `.abc` - approximately 40,000.

We then run the SIATEC algorithm on each MIDI file as follows. We have a function to run the algorithm once for a single MIDI file and a function to loop over all MIDI files, and a function to parse the output of a single run to a new .pat file.

In [18]:
def gather_patterns(lines):
    '''parses a .SIATECCompress file or similar. each pattern is on a line by itself
    in a format like:
    
    T(P(p(721,29),p(16081,29),p(31441,29),p(46801,29)),V(v(0,0),v(240,0),v(6480,2),v(7680,0),v(7920,0),v(9120,0),v(10320,2),v(10560,3),v(10800,4),v(11040,5),v(11280,6),v(11520,4),v(11760,3),v(12000,1),v(12240,2),v(12480,0),v(12720,-1),v(12960,-3),v(13200,-5),v(13440,-4),v(13680,-3),v(13920,-5),v(14160,-4)))

    and we will convert to the following format:
    
    (pattern, n_occurrences, occurences, length, complexity)
    
    where:
    
    pattern is a list of (time, pitch) pairs, starting at (0, 0) because we translate for comparability
    occurrences is a list of time values
    n_occurrences is the length of that list
    length is the number of pairs in pattern
    complexity is the number of unique pitches in the pattern, divided by its length
    
    ([(0, 0), (15360, 0), (30720, 0), (46080, 0)], 23, [0, 240, 6480, 7680, 7920, 9120, 10320, 10560, 10800, 11040, 11280, 11520, 11760, 12000, 12240, 12480, 12720, 12960, 13200, 13440, 13680, 13920, 14160], 4, 0.25)

    The MIDI time values look strange - but they are set algorithmically by abc2midi, 
    so corresponding notes should end up on corresponding time values.
    '''
    
    # these little fns are to help eval() to parse a pattern line (output by OMNISIA) by itself
    def T(P, V):
        return P, V
    def P(*ps):
        return ps
    def V(*vs):
        return vs
    def p(time, note):
        return (time, note)
    def v(time, note):
        return (time, note)
    for line in lines:
        # each pattern is in a line by itself
        # when we hit a blank line (length 1), there are no more patterns, so break
        if len(line) == 1: break
            
        # otherwise parse line with eval
        pattern, occurrences = eval(line)
        
        # we could set a threshold, ie minimum number of occurrences of pattern per tune,
        # but we haven't done that in other work, so we will not do it here
        # if len(occurrences) < 2: continue 
        
        # some basic stats
        length = len(pattern)
        complexity = len(set(n[1] for n in pattern)) / length
        
        # we normalise patterns so that all patterns start at (0, 0)
        # bearing in mind we use diatonic pitch.
        time0, pitch0 = pattern[0]
        pattern = [(pi[0] - time0, pi[1] - pitch0) for pi in pattern]
        locations = [occ[0] for occ in occurrences]
        #print('pattern', pattern)
        #print('locations', locations)
        #print('length', length)
        #print('complexity', complexity)
        
        yield pattern, len(locations), locations, length, complexity

In [17]:
def run_omnisia(filename, algo='SIATECCompress'):
    '''algo can be SIATECCompress (faster, overlapping patterns) or COSIATEC'''
    
    # we have to munge the filenames a bit:
    base = Path(filename).stem
    sufx = Path(filename).suffixes[0][1:]
    
    if algo == 'SIATECCompress': suffix = 'SIATECCompress'
    elif algo == 'COSIATEC': suffix = 'cos'
    else: raise ValueError(algo)
        
    output = f'test/{base}-{sufx}/{base}-diat.{suffix}'

    # -d true: morphetic (diatonic) pitch and so the filename below is 'diat', not 'chrom'
    # -o test: output directory
    # -nodate true: don't store date, avoid making unneeded directories
    # -min 3: no patterns less than 3
    cmd = f'java -Xmx1024M -jar OMNISIA.jar -a {algo} -d true -o test -nodate true -min 3 -i {filename}'.split(' ')
    print('running OMNISIA, this may take a few seconds...')
    subprocess.check_output(cmd)

    # we read the log file:
    # print('reading ' + output)
    L = open(output).readlines()
    return L

In [19]:
def run_omnisia_all(dirname):
    '''loop over MIDI files, run OMNISIA, and read its output, and write to .pat file'''
    i = 0
    start = time.time()
    failed = []
    for midiname in os.listdir(dirname):
        if not midiname.endswith('.mid'): continue
        patternname = midiname.replace('.mid', '.pat')
        
        # only run if the output doesn't exist
        # this is to help us run in batches, stop and restart
        if Path(os.path.join(dirname, patternname)).is_file(): continue 
           
        try:
            print('trying', midiname)
            L = run_omnisia(os.path.join(dirname, midiname))
            f = open(os.path.join(dirname, patternname), 'w')
            for pat in gather_patterns(L):
                f.write(pat+'\n')
            f.close()
        except:
            # an exception can arise such as an out of heap space memory error
            print(midiname, 'failed')
            failed.append(midiname) # just pass, try this file again later
        i += 1
        print(f'{i}: {midiname}: time {time.time() - start:.1f}s')
    return failed

In [50]:
# uncomment the line below to run OMNISIA on all .mid files - this will take hours or days
# failed = run_omnisia_all('thesession_dir')


A small number of `.mid` files are not processed correctly. It's likely that a larger heap space or some pre-processing could solve this issue. For now we can ignore it, as the number is very small relative to the size of the corpus.

In [21]:
len(failed)

[11117,
 11596,
 12549,
 12550,
 12932,
 13191,
 13386,
 13533,
 13686,
 14412,
 18740,
 18905,
 19084,
 22144,
 24190,
 24895,
 25594,
 26905,
 28603,
 29123,
 30598,
 31949,
 32095,
 32162,
 36120,
 3766,
 39369,
 40472,
 40736,
 42126,
 4357,
 6346,
 6447,
 7714,
 7881,
 'thesession11117.mid',
 'thesession11596.mid',
 'thesession14412.mid',
 'thesession18740.mid',
 'thesession22144.mid',
 'thesession24190.mid',
 'thesession28603.mid',
 'thesession30598.mid',
 'thesession32162.mid',
 'thesession39369.mid',
 'thesession40472.mid']

Next we read all the pat files, parsing them into a single data structure. We have a function that works for a single output (from a single tune), and a function that loops over all tunes.

In [24]:
def read_all_pat_files(dirname, test_sample=None):
    i = 0
    d = {}
    start = time.time()
    for patternname in os.listdir(dirname):
        if patternname.endswith('.pat'): 
            id = int(patternname[10:-4])
            # print(patternname, id)
            if id in d: continue
            try:
                d[id] = read_patterns(os.path.join(dirname, patternname))
            except ValueError:
                pass
            i += 1
        if test_sample is not None and i > test_sample:
            break
    return d

In [26]:
def read_patterns(filename):
    """Read a .pat file, which contains one pattern per line, already parsed"""
    lines = open(filename).readlines()
    results = []
    for line in lines:
        try:
            # a hack - in an old version of the code we saved just (pattern, n_occurrences)
            # but in the current version, we save (pattern, n_occurrences, occurrences, pattern_len, pattern_complexity)
            # Only (pattern, n_occurrences) are needed for our current project. To avoid a large re-run
            # we code this function to accept either format.
            pattern, n_occurrences, occurrences, pattern_len, pattern_complexity = eval(line)
        except:
            pattern, n_occurrences = eval(line)
        results.append((pattern, n_occurrences))
    return results

In [52]:
# uncomment this line to read in all the .pat files to a data structure
# d, failed = read_omnisia_outputs('thesession_dir')

# and save to a pkl
# f = open('thesession_omnisia_patterns.pkl', 'wb')
# pickle.dump(d, f)

thesession1.pat 1
thesession10.pat 10
thesession1000.pat 1000
thesession10000.pat 10000
thesession10001.pat 10001
thesession10002.pat 10002
thesession10003.pat 10003
thesession10004.pat 10004
thesession10005.pat 10005
thesession10006.pat 10006
thesession10007.pat 10007
thesession10008.pat 10008
thesession10009.pat 10009
thesession1001.pat 1001
thesession10010.pat 10010
thesession10011.pat 10011
thesession10012.pat 10012
thesession10013.pat 10013
thesession10014.pat 10014
thesession10015.pat 10015
thesession10016.pat 10016
thesession10017.pat 10017
thesession10018.pat 10018
thesession1002.pat 1002
thesession10020.pat 10020
thesession10021.pat 10021
thesession10022.pat 10022
thesession10023.pat 10023
thesession10024.pat 10024
thesession10025.pat 10025
thesession10026.pat 10026
thesession10027.pat 10027
thesession10028.pat 10028
thesession10029.pat 10029
thesession1003.pat 1003
thesession10030.pat 10030
thesession10031.pat 10031
thesession10032.pat 10032
thesession10033.pat 10033
thesessi

thesession10480.pat 10480
thesession10481.pat 10481
thesession10482.pat 10482
thesession10483.pat 10483
thesession10486.pat 10486
thesession10487.pat 10487
thesession10488.pat 10488
thesession1049.pat 1049
thesession10490.pat 10490
thesession10491.pat 10491
thesession10492.pat 10492
thesession10493.pat 10493
thesession10494.pat 10494
thesession10495.pat 10495
thesession10496.pat 10496
thesession10498.pat 10498
thesession10499.pat 10499
thesession105.pat 105
thesession1050.pat 1050
thesession10500.pat 10500
thesession10501.pat 10501
thesession10502.pat 10502
thesession10503.pat 10503
thesession10504.pat 10504
thesession10505.pat 10505
thesession10506.pat 10506
thesession10507.pat 10507
thesession10508.pat 10508
thesession10509.pat 10509
thesession1051.pat 1051
thesession10510.pat 10510
thesession10511.pat 10511
thesession10512.pat 10512
thesession10513.pat 10513
thesession10514.pat 10514
thesession10515.pat 10515
thesession10516.pat 10516
thesession10517.pat 10517
thesession10518.pat 10

thesession10893.pat 10893
thesession10894.pat 10894
thesession10895.pat 10895
thesession10896.pat 10896
thesession10897.pat 10897
thesession10898.pat 10898
thesession10899.pat 10899
thesession109.pat 109
thesession1090.pat 1090
thesession10900.pat 10900
thesession10901.pat 10901
thesession10902.pat 10902
thesession10904.pat 10904
thesession10905.pat 10905
thesession10906.pat 10906
thesession10907.pat 10907
thesession10908.pat 10908
thesession10909.pat 10909
thesession1091.pat 1091
thesession10910.pat 10910
thesession10911.pat 10911
thesession10912.pat 10912
thesession10913.pat 10913
thesession10914.pat 10914
thesession10916.pat 10916
thesession10917.pat 10917
thesession10918.pat 10918
thesession10919.pat 10919
thesession1092.pat 1092
thesession10920.pat 10920
thesession10921.pat 10921
thesession10922.pat 10922
thesession10923.pat 10923
thesession10924.pat 10924
thesession10925.pat 10925
thesession10926.pat 10926
thesession10927.pat 10927
thesession10928.pat 10928
thesession10929.pat 10

thesession11215.pat 11215
thesession11216.pat 11216
thesession11217.pat 11217
thesession11218.pat 11218
thesession11219.pat 11219
thesession1122.pat 1122
thesession11220.pat 11220
thesession11221.pat 11221
thesession11222.pat 11222
thesession11223.pat 11223
thesession11224.pat 11224
thesession11225.pat 11225
thesession11226.pat 11226
thesession11227.pat 11227
thesession11228.pat 11228
thesession11229.pat 11229
thesession1123.pat 1123
thesession11230.pat 11230
thesession11231.pat 11231
thesession11232.pat 11232
thesession11233.pat 11233
thesession11234.pat 11234
thesession11235.pat 11235
thesession11236.pat 11236
thesession11237.pat 11237
thesession11238.pat 11238
thesession11239.pat 11239
thesession1124.pat 1124
thesession11241.pat 11241
thesession11242.pat 11242
thesession11243.pat 11243
thesession11244.pat 11244
thesession11245.pat 11245
thesession11246.pat 11246
thesession11247.pat 11247
thesession11248.pat 11248
thesession11249.pat 11249
thesession1125.pat 1125
thesession11250.pat 

thesession11532.pat 11532
thesession11533.pat 11533
thesession11534.pat 11534
thesession11535.pat 11535
thesession11536.pat 11536
thesession11537.pat 11537
thesession11538.pat 11538
thesession11539.pat 11539
thesession1154.pat 1154
thesession11540.pat 11540
thesession11541.pat 11541
thesession11543.pat 11543
thesession11545.pat 11545
thesession11546.pat 11546
thesession11547.pat 11547
thesession11548.pat 11548
thesession11549.pat 11549
thesession1155.pat 1155
thesession11551.pat 11551
thesession11552.pat 11552
thesession11553.pat 11553
thesession11554.pat 11554
thesession11556.pat 11556
thesession11557.pat 11557
thesession11558.pat 11558
thesession11559.pat 11559
thesession1156.pat 1156
thesession11560.pat 11560
thesession11561.pat 11561
thesession11562.pat 11562
thesession11563.pat 11563
thesession11564.pat 11564
thesession11565.pat 11565
thesession11566.pat 11566
thesession11568.pat 11568
thesession11569.pat 11569
thesession1157.pat 1157
thesession11570.pat 11570
thesession11571.pat 

...

thesession8492.pat 8492
thesession8494.pat 8494
thesession8495.pat 8495
thesession8496.pat 8496
thesession8498.pat 8498
thesession8499.pat 8499
thesession85.pat 85
thesession850.pat 850
thesession8500.pat 8500
thesession8501.pat 8501
thesession8504.pat 8504
thesession8505.pat 8505
thesession8507.pat 8507
thesession8508.pat 8508
thesession8509.pat 8509
thesession851.pat 851
thesession8510.pat 8510
thesession8511.pat 8511
thesession8512.pat 8512
thesession8513.pat 8513
thesession8514.pat 8514
thesession8515.pat 8515
thesession8516.pat 8516
thesession8517.pat 8517
thesession8518.pat 8518
thesession8519.pat 8519
thesession852.pat 852
thesession8520.pat 8520
thesession8521.pat 8521
thesession8522.pat 8522
thesession8523.pat 8523
thesession8524.pat 8524
thesession8525.pat 8525
thesession8526.pat 8526
thesession8527.pat 8527
thesession8528.pat 8528
thesession8529.pat 8529
thesession853.pat 853
thesession8530.pat 8530
thesession8531.pat 8531
thesession8532.pat 8532
thesession8533.pat 8533
thes

thesession8834.pat 8834
thesession8835.pat 8835
thesession8836.pat 8836
thesession8837.pat 8837
thesession8838.pat 8838
thesession8839.pat 8839
thesession884.pat 884
thesession8841.pat 8841
thesession8842.pat 8842
thesession8843.pat 8843
thesession8844.pat 8844
thesession8845.pat 8845
thesession8846.pat 8846
thesession8847.pat 8847
thesession8848.pat 8848
thesession8849.pat 8849
thesession885.pat 885
thesession8851.pat 8851
thesession8852.pat 8852
thesession8853.pat 8853
thesession8854.pat 8854
thesession8855.pat 8855
thesession8857.pat 8857
thesession8858.pat 8858
thesession8859.pat 8859
thesession886.pat 886
thesession8860.pat 8860
thesession8861.pat 8861
thesession8862.pat 8862
thesession8863.pat 8863
thesession8865.pat 8865
thesession8866.pat 8866
thesession8867.pat 8867
thesession8868.pat 8868
thesession8869.pat 8869
thesession887.pat 887
thesession8870.pat 8870
thesession8872.pat 8872
thesession8873.pat 8873
thesession8874.pat 8874
thesession8875.pat 8875
thesession8876.pat 8876


thesession9220.pat 9220
thesession9221.pat 9221
thesession9222.pat 9222
thesession9223.pat 9223
thesession9224.pat 9224
thesession9225.pat 9225
thesession9226.pat 9226
thesession9227.pat 9227
thesession9228.pat 9228
thesession9229.pat 9229
thesession923.pat 923
thesession9230.pat 9230
thesession9231.pat 9231
thesession9232.pat 9232
thesession9233.pat 9233
thesession9234.pat 9234
thesession9235.pat 9235
thesession9236.pat 9236
thesession9237.pat 9237
thesession9238.pat 9238
thesession924.pat 924
thesession9240.pat 9240
thesession9243.pat 9243
thesession9245.pat 9245
thesession9246.pat 9246
thesession9247.pat 9247
thesession9248.pat 9248
thesession9249.pat 9249
thesession925.pat 925
thesession9250.pat 9250
thesession9251.pat 9251
thesession9252.pat 9252
thesession9253.pat 9253
thesession9254.pat 9254
thesession9255.pat 9255
thesession9256.pat 9256
thesession9257.pat 9257
thesession9258.pat 9258
thesession9259.pat 9259
thesession926.pat 926
thesession9260.pat 9260
thesession9261.pat 9261


thesession9563.pat 9563
thesession9564.pat 9564
thesession9565.pat 9565
thesession9566.pat 9566
thesession9567.pat 9567
thesession9568.pat 9568
thesession9569.pat 9569
thesession957.pat 957
thesession9570.pat 9570
thesession9571.pat 9571
thesession9572.pat 9572
thesession9573.pat 9573
thesession9574.pat 9574
thesession9577.pat 9577
thesession9578.pat 9578
thesession9579.pat 9579
thesession958.pat 958
thesession9580.pat 9580
thesession9581.pat 9581
thesession9583.pat 9583
thesession9584.pat 9584
thesession9585.pat 9585
thesession9586.pat 9586
thesession9587.pat 9587
thesession9588.pat 9588
thesession9589.pat 9589
thesession959.pat 959
thesession9591.pat 9591
thesession9592.pat 9592
thesession9593.pat 9593
thesession9594.pat 9594
thesession9595.pat 9595
thesession9596.pat 9596
thesession9597.pat 9597
thesession9598.pat 9598
thesession9599.pat 9599
thesession96.pat 96
thesession960.pat 960
thesession9600.pat 9600
thesession9601.pat 9601
thesession9602.pat 9602
thesession9603.pat 9603
thes

thesession994.pat 994
thesession9940.pat 9940
thesession9941.pat 9941
thesession9942.pat 9942
thesession9943.pat 9943
thesession9944.pat 9944
thesession9945.pat 9945
thesession9946.pat 9946
thesession9947.pat 9947
thesession9948.pat 9948
thesession9949.pat 9949
thesession995.pat 995
thesession9950.pat 9950
thesession9951.pat 9951
thesession9952.pat 9952
thesession9953.pat 9953
thesession9954.pat 9954
thesession9957.pat 9957
thesession9958.pat 9958
thesession9959.pat 9959
thesession996.pat 996
thesession9961.pat 9961
thesession9962.pat 9962
thesession9963.pat 9963
thesession9964.pat 9964
thesession9965.pat 9965
thesession9966.pat 9966
thesession9968.pat 9968
thesession9969.pat 9969
thesession997.pat 997
thesession9970.pat 9970
thesession9972.pat 9972
thesession9974.pat 9974
thesession9975.pat 9975
thesession9976.pat 9976
thesession9977.pat 9977
thesession9978.pat 9978
thesession9979.pat 9979
thesession998.pat 998
thesession9980.pat 9980
thesession9984.pat 9984
thesession9985.pat 9985
th

Above, in two places, we have commented-out lines which launch large runs. Instead, for this notebook, the user can load the resulting data directly as a pickle:

In [ ]:
f = open('thesession_omnisia_patterns.pkl', 'rb')
d = pickle.load(f)

The pattern data is not very large in memory:

In [54]:
import sys
sys.getsizeof(d)

1310800

### Part 2: using patterns for tune family detection

Next, we can proceed to use our pattern data `d` for tune family detection.

We begin with ground truth, a set of tune family annotations described by Danny Diamond (Deliverables D5.5-5.6, and MSc thesis, 2024). The tunes in these annotations are from The Session, so their patterns have already been processed above.

In [51]:
annotated = pd.read_csv('thesession_subset_metadata_with_tune_family_annotation.csv')

There are 315 members in 10 tune families:

In [52]:
len(annotated)

315

In [54]:
set(annotated['tune_family'])

{'Blackbird',
 'Drowsy Maggie',
 'Gilderoy',
 "Greig's Pipes",
 'Hob or Nob',
 "Jenny's Welcome to Charlie",
 'Johnny Cope',
 "Lord McDonald's",
 "O'Sullivan's March",
 'Road to Lisdoonvarna'}

## Tune similarity via patterns



In [14]:
def get_pats(tune):
    '''Given a tune represented by a list of (pattern, n_occurrences) values,
    take only the non-trivial patterns and return them as a set.'''
    s = set()
    for pat, n_occurrences in tune:
        # complexity = len(set(el[1] for el in pat)) / len(pat)
        unique_notes = len(set(el[1] for el in pat))
        if unique_notes > 1:
            s.add(tuple(pat))
    return s
    
def compare_patterns(tune_i, tune_j):
    '''Check for an intersection among the non-trivial patterns of two tunes
    (each represented by a list of (pattern, n_occurrences) values as above.)'''
    pi = get_pats(tune_i)
    pj = get_pats(tune_j)
    intersect = pi.intersection(pj)
    if intersect: return True
    else: return False

Next we will go through all of the 315 annotated tune-family members, and for each of them
we'll find which of the 40k The Session tunes it has any patterns in common with. 

In [36]:
annotated_ids = list(annotated['identifiers'])

In [35]:
links = defaultdict(list)
k = 0
for i in annotated_ids: # just the 315 annotated tune-family members
    print(f'working on annotated id {i} ({k}/{len(annotated_ids)})')
    for j in d: # all 40k The Session members
        if i == j: continue
        if compare_patterns(d[i], d[j]):
            links[i].append(j)
    k += 1


working on annotated id 1029 (0/315)
working on annotated id 14252 (1/315)
working on annotated id 25163 (2/315)
working on annotated id 27315 (3/315)
working on annotated id 27316 (4/315)
working on annotated id 27842 (5/315)
working on annotated id 29508 (6/315)
working on annotated id 39472 (7/315)
working on annotated id 75 (8/315)
working on annotated id 12552 (9/315)
working on annotated id 12553 (10/315)
working on annotated id 12554 (11/315)
working on annotated id 12556 (12/315)
working on annotated id 12557 (13/315)
working on annotated id 12558 (14/315)
working on annotated id 12559 (15/315)
working on annotated id 12560 (16/315)
working on annotated id 21434 (17/315)
working on annotated id 23677 (18/315)
working on annotated id 24548 (19/315)
working on annotated id 24549 (20/315)
working on annotated id 24681 (21/315)
working on annotated id 25452 (22/315)
working on annotated id 25695 (23/315)
working on annotated id 26044 (24/315)
working on annotated id 26249 (25/315)


working on annotated id 12410 (209/315)
working on annotated id 12411 (210/315)
working on annotated id 12412 (211/315)
working on annotated id 12413 (212/315)
working on annotated id 12414 (213/315)
working on annotated id 12415 (214/315)
working on annotated id 20867 (215/315)
working on annotated id 21232 (216/315)
working on annotated id 23223 (217/315)
working on annotated id 23242 (218/315)
working on annotated id 23248 (219/315)
working on annotated id 24826 (220/315)
working on annotated id 27945 (221/315)
working on annotated id 27946 (222/315)
working on annotated id 29583 (223/315)
working on annotated id 31098 (224/315)
working on annotated id 31476 (225/315)
working on annotated id 35491 (226/315)
working on annotated id 37487 (227/315)
working on annotated id 39017 (228/315)
working on annotated id 39098 (229/315)
working on annotated id 39125 (230/315)
working on annotated id 787 (231/315)
working on annotated id 13917 (232/315)
working on annotated id 13918 (233/315)
wo

Next, our method is to calculate the degree of similarity as *intersection divided by union* (IOU) between two sets of patterns.

In [38]:
def intersection_over_union(s1, s2):
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [39]:
def run_iou_all():
    iou = {}
    for tune_i in annotated_ids:
        pats_i = get_pats(d[tune_i])
        for tune_j in links[tune_i]:
            iou[(tune_i, tune_j)] = intersection_over_union(pats_i, get_pats(d[tune_j]))
    return iou


# as before we comment-out a long-running computation:
# iou = run_iou_all()

# after running that, we would save the IOU data to disk
# f = open('thesession_annotated_omnisia_iou.pkl', 'wb')
# pickle.dump(iou, f)


In [55]:
# alternative: in case you want to avoid running IOU, which takes some time, 
# you can instead load a dump of the IOU results as below:
f = open('thesession_annotated_omnisia_iou.pkl', 'rb')
iou = pickle.load(f)

Now we are ready to produce results and look at performance. For each tune in the ground-truth tune family annotations, we rank all of the other tunes in The Session which have any patterns in common.
We rank them by IOU in decreasing order. Larger IOU values indicate higher similarity.

We simply count how many of the top 10 in that ranking are correct, ie are in the same tune family.

In [56]:
hit_results = {}
for tune_i in annotated_ids:
    sim = sorted([(iou[(tune_i, tune_j)], tune_j) for tune_j in links[tune_i]], reverse=True)
    # print(sim)
    hits = 0
    for simval, tune_j in sim[:10]:
        if tune_j in annotated_ids: # TODO is this correct?
            hits += 1
    hit_results[tune_i] = hits / 10

We see results are usually 0.0-0.3, ie 0-3 out of the top 10.

In [57]:
hit_results

{1029: 0.0,
 14252: 0.1,
 25163: 0.0,
 27315: 0.1,
 27316: 0.0,
 27842: 0.1,
 29508: 0.0,
 39472: 0.0,
 75: 0.1,
 12552: 0.0,
 12553: 0.1,
 12554: 0.0,
 12556: 0.1,
 12557: 0.0,
 12558: 0.0,
 12559: 0.0,
 12560: 0.0,
 21434: 0.1,
 23677: 0.1,
 24548: 0.0,
 24549: 0.0,
 24681: 0.3,
 25452: 0.0,
 25695: 0.1,
 26044: 0.0,
 26249: 0.0,
 29406: 0.1,
 31921: 0.0,
 35686: 0.1,
 42137: 0.0,
 29348: 0.1,
 29515: 0.1,
 31916: 0.0,
 32702: 0.0,
 1805: 0.2,
 27795: 0.0,
 33355: 0.0,
 33356: 0.1,
 36399: 0.0,
 38698: 0.0,
 2479: 0.1,
 15787: 0.0,
 35452: 0.0,
 1104: 0.1,
 1996: 0.0,
 4101: 0.1,
 4508: 0.0,
 14355: 0.0,
 14356: 0.1,
 14357: 0.1,
 15415: 0.0,
 17110: 0.0,
 17111: 0.0,
 17112: 0.0,
 24050: 0.0,
 26494: 0.0,
 27198: 0.0,
 28174: 0.0,
 28821: 0.1,
 31201: 0.0,
 32399: 0.0,
 34225: 0.0,
 36690: 0.1,
 39782: 0.0,
 27066: 0.0,
 249: 0.0,
 250: 0.2,
 12965: 0.0,
 12966: 0.2,
 12967: 0.1,
 12968: 0.1,
 12969: 0.2,
 12970: 0.1,
 12971: 0.0,
 12972: 0.2,
 21776: 0.0,
 25089: 0.2,
 27843: 0.2,


In [48]:
#f = open('thesession_annotated_omnisia_precision10_results.pkl', 'wb')
#pickle.dump(map_results, f)

We can compare the results (in text format, above) to the results achieved by Diamond (reported in Polifonia deliverables D5.5-5.6 and MSc thesis, 2024). Each result is effectively the number of 'hits' in the top 10 predictions, for a given query tune. As shown in the image below, the 'motif' method of Diamond sometimes gives a low result - 0 to 3 out of 10 - but often achieves values above 5 out of 10, never achieved using SIATEC patterns.

![](img/motif_map_per_tune.png)

### Discussion and Conclusion

Meredith (2013) writes:
"Both algorithms [COSIATEC and SIATECCompress] are founded on the hypothesis that the
best ways of understanding a piece of music are those that
are represented by the shortest descriptions of the piece. In
other words, they are designed to explore the notion that
music analysis is effectively just music compression."

This hypothesis is central to my (McDermott) view of music analysis also. I look to papers such as Schmidhuber's *Low-Complexity Art*. In related work (working with Maziar Kanani and Seán O'Leary, not part of the Polifonia project), I use the *Assembly Pathway* (Cronin and colleagues) and the *Sequitur* algorithm (Nevil-Manning and Witten).

However, it seems that a compression approach is not necessarily well-suited to the task of *linking pieces of music*. This difference in goals likely explains the unexpected results.
